In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
ind2label = {0: 'Cep', 1: 'Cep_1', 2: 'Cep_12', 3: 'Cep_2', 4: 'Cep_F', 5: 'Cep_F1', 6: 'DCep', 7: 'DPV', 8: 'DSCT', 9: 'DSCT_F1', 10: 'DSCT_S', 11: 'LPV', 12: 'LPV_Mira', 13: 'LPV_OSARG', 14: 'LPV_SRV', 15: 'RRLyr', 16: 'RRLyr_RRab', 17: 'RRLyr_RRc', 18: 'RRLyr_RRd', 19: 'RRLyr_RRe', 20: 'T2Cep', 21: 'T2Cep_BLHer', 22: 'T2Cep_RVTau', 23: 'T2Cep_WVir', 24: 'VAR', 25: 'aCep', 26: 'aCep_F', 27: 'eclipsing', 28: 'mCep', 29: 'pulsating'}
#ogle new data, used labels
ind2label2 = {0: 'Cep_1', 1: 'Cep_12', 2: 'Cep_2', 3: 'Cep_F', 4: 'Cep_F1', 5: 'DPV', 6: 'DSCT_F1', 7: 'DSCT_S', 8: 'LPV_Mira', 9: 'LPV_OSARG', 10: 'LPV_SRV', 11: 'RRLyr_RRab', 12: 'RRLyr_RRc', 13: 'RRLyr_RRd', 14: 'RRLyr_RRe', 15: 'T2Cep_BLHer', 16: 'T2Cep_RVTau', 17: 'T2Cep_WVir', 18: 'aCep_F'}
def cm(predtruth, seperator, labels='all'):
    pred, truth = [], []
    for i in predtruth[0][:seperator][0]:
        for j in i: 
            if labels=='used':
                pred.append(ind2label2[j])
            else:
                pred.append(ind2label[j])
    for i in predtruth[0][seperator:seperator+1][0]:
        for j in i: 
            if labels=='used':
                truth.append(ind2label2[j])
            else:
                truth.append(ind2label[j])
    y_actu = pd.Series(truth, name='Actual')
    y_pred = pd.Series(pred, name='Predicted')
    return pd.crosstab(y_actu, y_pred, normalize='index')

def plot_confusion_matrix(df_confusion, title='Confusion matrix', cmap=plt.cm.gray_r):
    plt.matshow(df_confusion, cmap=cmap) # imshow
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)

def make_table(eval):
    tanimoto, heights, pathlengths, similarities, true_similarities = eval[0][:8],eval[0][8:16],eval[0][16:24],eval[0][24:32],eval[0][32:40]
    metric = ['tanimoto','height','pathlength','similarity','true_similarity']
    data = [tanimoto, heights, pathlengths, similarities, true_similarities]
    data = np.asarray(data).reshape((5,8))
    df=pd.DataFrame(columns=['mean', 'med', 'min','max','1','5','95','99'], data=data)
    df.insert(loc=0, column='metric', value=metric)
    return df

from collections import OrderedDict
def find_sims(probs, truth, alllabels=True, cos=False):
    label2int, int2label, unique_labels = {}, {}, OrderedDict()
    with open('data/ogle_class_names.txt') as class_list:
    for c in class_list:  
        num, classname = c.strip().split()
        label2int[classname] = int(num) 
        int2label[int(num)] = classname
        unique_labels[classname] = None 
    with open('data/ogle.unitsphere.pickle', 'rb') as pf:
        embedding = pickle.load(pf)
        linear_labels = embedding['label2ind'] 
        embed_labels = embedding['ind2label']
        embedding = embedding['embedding']

unique_label, count = np.unique(list(unique_labels.keys()), return_counts=True) 
convert_label = dict(zip(unique_label, np.arange(len(unique_label))))
convert_emb2ind = lambda i: convert_label[int2label[embed_labels[i]]] 
i2l = ind2label if alllabels else ind2label2

def find_similarity(prediction, ground_truth): 
    return np.matmul(embedding[linear_labels[label2int[i2l[prediction]]]].T,embedding[linear_labels[label2int[i2l[ground_truth]]]])

sim_list = []
for p in range(len(probs)):
    all_sims = [] 
    for i in range(len(probs[p])):
        max_index, max_value = max(enumerate(probs[p]), key=lambda pair: pair[1])
        probs[p][max_index] = -1
        if cos: 
            pre = convert_emb2ind(max_index)
        else: 
            pre = max_index
        sim = find_similarity(pre, truth[p])
        all_sims.append(sim)
    sim_list.append(all_sims)
    mean_sim = np.mean(sim_list, axis=0)
    return mean_sim

In [3]:
h_pred = np.load('results_pred.npy', allow_pickle=True)
h_truth = np.load('results_truth.npy', allow_pickle=True)

In [4]:
h_pred

array([ 15,  36,  18, 101,  89,  91, 113,  19, 118,  84,  94, 110,  39,
       114,  35,   7,  53,  70,  89,  19,  98,  54,  97,  26,  31, 114,
        88, 114,  15,  47, 118,  29,  68, 114,  35,  29,  94,  89,  17,
        80,  53,  80,  31,  57, 101,  32,  68,  44,  94,  15,  94,  68,
        19,  81,  66,  68,  35,  98,  98,  44,  38,  19,  61,  73])

In [ ]:
h_df_confusion = cm(h_predtruth, 1)

In [ ]:
fig, (ax1) = plt.subplots(1,1,figsize=(23,7))
g1 = sn.heatmap(h_df_confusion, annot=True)
ax1.set_title("New Loss, Hidden")
plt.show()